In [ ]:
import sys
sys.path.insert(0, '..')
import time
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib notebook  
import numpy as np


from Configuration import Configuration
from MotorUnitPool import MotorUnitPool
from AfferentPool import AfferentPool
from SynapsesFactory import SynapsesFactory

In [ ]:
Nsim = 15
FirstStim = 9
LastStim = 20.0

In [ ]:
conf = Configuration('confH-ReflexM-WaveCurve.rmto')

In [ ]:
print Nsim
print FirstStim
print LastStim

In [ ]:
t = np.arange(0.0, conf.simDuration_ms, conf.timeStep_ms)

Mp = np.zeros((int(Nsim), 1))
Hp = np.zeros((int(Nsim), 1))
Stim = np.zeros((int(Nsim), 1))
emg = np.zeros([len(t), int(Nsim)])
pools = dict()
pools[0] = MotorUnitPool(conf, 'SOL')
pools[1] = AfferentPool(conf,'Ia', 'SOL')
pools[2]= AfferentPool(conf,'Ia', 'LG')
Syn = SynapsesFactory(conf, pools)
del Syn 

In [ ]:
for j  in xrange(0, int(Nsim)):
    Stim[j] = FirstStim + j * (LastStim - FirstStim) / (Nsim - 1)
    conf.changeConfigurationParameter('stimIntensity_PTN', FirstStim + j * (LastStim - FirstStim) / (Nsim - 1),'')
    for k in xrange(0, len(pools)):
        for i in xrange(0,len(pools[k].unit)):
            pools[k].unit[i].createStimulus()
    tic = time.time()
    for i in xrange(0, len(t)-1):
        pools[0].atualizeMotorUnitPool(t[i])
        pools[1].atualizeAfferentPool(t[i],0)
        pools[2].atualizeAfferentPool(t[i],0)
    toc = time.time()
    print str(toc - tic) + ' seconds'
    pools[0].getMotorUnitPoolEMG()
    Mp[j] = np.max(pools[0].emg[t<20]) - np.min(pools[0].emg[t<20])
    Hp[j] = np.max(pools[0].emg[t>20]) - np.min(pools[0].emg[t>20])
    print Mp[j]
    print Hp[j]
    emg[:,[j]] = pools[0].emg
    pools[0].listSpikes()
    pools[1].listSpikes()
    pools[2].listSpikes()
    sp1 = pools[0].poolTerminalSpikes
    sp2 = pools[1].poolTerminalSpikes
    sp2 = pools[2].poolTerminalSpikes
    for k in xrange(0, len(pools)):
        pools[k].reset()

In [ ]:

f1 = plt.figure()
plt.plot(Stim, Hp, Stim, Mp)


In [ ]:
np.max(Hp)/np.max(Mp)

In [ ]:
Mp

In [ ]:
f2 = plt.figure()
plt.plot(t, emg[:,14])


In [ ]:
pools[0].listSpikes()

In [ ]:
f3 = plt.figure()
plt.plot(sp1[:, 0],
         sp1[:, 1]+1, '.')
plt.xlabel('t (ms)')
plt.ylabel('Motor Unit index')


In [ ]:
pools[0].poolTerminalSpikes[:, 1]

In [2]:
%load_ext fortranmagic


/home/rnwatanabe/miniconda3/lib/python3.6/site-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


In [3]:
%%fortran -vvv --fcompiler gnu95

module HReflexMWave
    use MotorUnitPoolClass
    use NeuralTractClass
    use InterneuronPoolClass
    use SynapticNoiseClass
    use ConfigurationClass
    use ogpf 
    use randomSeedInitialize
    use SynapsesFactoryModule
    use jointAnkleForceTaskClass
    use AfferentPoolClass
    implicit none 
    
            
    contains
    subroutine executeProg()
        type(Configuration), target :: conf
        real(wp), parameter :: pi = 4 * atan(1.0_wp)    
        real(wp) :: dt
        real(wp) :: tf
        integer :: timeLength
        integer :: i, j, k
        real(wp), dimension(:), allocatable :: t
        real(wp) :: tic, toc
        type(gpf) :: gp
        character(len = 80) :: pool, muscle, paramTag
        character(len = 80) :: filename = 'confH-ReflexM-WaveCurve.rmto'
        type(MotorUnitPool), dimension(:), allocatable, target :: motorUnitPools
        type(NeuralTract), dimension(:), allocatable :: neuralTractPools    
        type(InterneuronPool), dimension(:), allocatable, target :: interneuronPools    
        type(SynapticNoise), dimension(:), allocatable:: synapticNoisePools   
        type(AfferentPool), dimension(:), allocatable:: afferentPools    
        type(jointAnkleForceTask) :: ankle
        real(wp) :: angle
        real(wp) , dimension(200):: Amp, phase
        integer :: Nsim
        real(wp) :: FirstStim, LastStim
        real(wp), dimension(:), allocatable :: Mp, Hp, Stim
        real(wp), dimension(:,:), allocatable :: emg
        character(len = 80) :: value1, value2
        real(wp) :: spindleFR
        character(len = 3) :: fileNumber

        Nsim = 35
        FirstStim = 9.0
        LastStim = 25

        call init_random_seed()

        conf = Configuration(filename)

        tf = conf%simDuration_ms
        dt = conf%timeStep_ms
        timeLength = int(tf/dt)

        allocate(t(timeLength))

        t = [(dt*(i-1), i=1, timeLength)]

        allocate(emg(timeLength, Nsim))
        allocate(Mp(Nsim))
        allocate(Hp(Nsim))
        allocate(Stim(Nsim))   



        allocate(afferentPools(2))
        pool = 'Ia'
        muscle = 'SOL'
        afferentPools(1) = AfferentPool(conf, pool, muscle)

        pool = 'Ia'
        muscle = 'LG'
        afferentPools(2) = AfferentPool(conf, pool, muscle)

        allocate(neuralTractPools(0))

        allocate(motorUnitPools(1))
        pool = 'SOL'
        motorUnitPools(1) = MotorUnitPool(conf, pool)    


        ankle = jointAnkleForceTask(conf, motorUnitPools)
        allocate(interneuronPools(0))

        synapticNoisePools = synapseFactory(conf, neuralTractPools, &
                                            motorUnitPools, &
                                            interneuronPools, &
                                            afferentPools)


        do j = 1, Nsim
            Stim(j) = FirstStim + j * (LastStim - FirstStim) / (Nsim - 1.0)
            paramTag = 'stimIntensity_PTN'
            write(value1, '(F15.6)')Stim(j)
            value2 = ''
            call conf%changeConfigurationParameter(paramTag, value1,value2)

            do i = 1, size(motorUnitPools(1)%unit)
                call motorUnitPools(1)%unit(i)%createStimulus()
            end do

            do i = 1, size(afferentPools(1)%unit)
                call afferentPools(1)%unit(i)%createStimulus()
            end do
            do i = 1, size(afferentPools(2)%unit)
                call afferentPools(2)%unit(i)%createStimulus()
            end do


            spindleFR = 0.0
            call cpu_time(tic)
            do i = 1, timeLength
                call motorUnitPools(1)%atualizeMotorUnitPool(t(i))
                call afferentPools(1)%atualizeAfferentPool(t(i),spindleFR)
                call afferentPools(2)%atualizeAfferentPool(t(i),spindleFR)
            end do
            call cpu_time(toc)
            print '(F15.6, A)', toc - tic, ' seconds'
            call motorUnitPools(1)%getMotorUnitPoolEMG()
            Mp(j) = maxval(motorUnitPools(1)%emg(1:500)) - minval(motorUnitPools(1)%emg(1:500))
            Hp(j) = maxval(motorUnitPools(1)%emg(501:)) - minval(motorUnitPools(1)%emg(501:))
            print '(F15.6)', Mp(j)
            print '(F15.6)', Hp(j)
            emg(:,j) = motorUnitPools(1)%emg
            call motorUnitPools(1)%listSpikes()
            call afferentPools(1)%listSpikes()
            call afferentPools(2)%listSpikes()
            write(fileNumber, '(I2)')j
            filename = 'spikes'// fileNumber //'.txt'
            open(1, file=filename, status = 'replace') 

            do i = 1, size(motorUnitPools(1)%poolTerminalSpikes, 1)            
                write(1, '(F15.6, 1X, F15.1)') motorUnitPools(1)%poolTerminalSpikes(i,1),&
                            motorUnitPools(1)%poolTerminalSpikes(i,2)
            end do
            close(1)

            call motorUnitPools(1)%reset()
            call afferentPools(1)%reset()
            call afferentPools(2)%reset()
        end do

        call gp%title('EMG')
        call gp%xlabel('t (ms))')
        call gp%ylabel('emg (mV)')
        call gp%plot(t, emg(:,10),  'with line lw 2 lc rgb "#0008B0"')

        call gp%title('Hp')
        call gp%xlabel('Stimulus intensity (mA)')
        call gp%ylabel('EMG peak (mV)')
        call gp%plot(Stim, Hp, 'with line lw 2 lc rgb "#0008B0"')

        call gp%title('Mp')
        call gp%xlabel('Stimulus intensity (mA)')
        call gp%ylabel('EMG peak (mV)')
        call gp%plot(Stim, Mp, 'with line lw 2 lc rgb "#0008B0"')

        call gp%title('Mp-Hp')
        call gp%xlabel('Stimulus intensity (mA)')
        call gp%ylabel('EMG peak (mV)')
        call gp%plot(x1=Stim, y1=Mp, ls1='with line lw 2 lc rgb "blue"', &
                     x2 = Stim, y2 = Hp, ls2='with line lw 2 lc rgb "red"')

        print '(F15.6)', maxval(Hp)/maxval(Mp)   
    end subroutine

    
end module HReflexMWave

Running...
   /home/rnwatanabe/miniconda3/bin/python -m numpy.f2py --fcompiler=gnu95 -m _fortran_magic_5806b92bea7a495699e54450c5bb35c5 -c /home/rnwatanabe/.cache/ipython/fortran/_fortran_magic_5806b92bea7a495699e54450c5bb35c5.f90
running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "_fortran_magic_5806b92bea7a495699e54450c5bb35c5" sources
f2py options: []
f2py:> /tmp/tmp_elg9dwo/src.linux-x86_64-3.6/_fortran_magic_5806b92bea7a495699e54450c5bb35c5module.c
creating /tmp/tmp_elg9dwo/src.linux-x86_64-3.6
Reading fortran codes...
	Reading file '/home/rnwatanabe/.cache/ipython/fortran/_fortran_magic_5806b92bea7a495699e54450c5bb35c5.f90' (format:free)
Line #99 in /home/rnwatanabe/.cache/ipython/fortran/_fortran_magic_5806b92bea7a495699e54450c5bb35c5.f90:"            call conf%change

Traceback (most recent call last):
  File "/home/rnwatanabe/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/rnwatanabe/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/rnwatanabe/miniconda3/lib/python3.6/site-packages/numpy/f2py/__main__.py", line 27, in <module>
    main()
  File "/home/rnwatanabe/miniconda3/lib/python3.6/site-packages/numpy/f2py/f2py2e.py", line 648, in main
    run_compile()
  File "/home/rnwatanabe/miniconda3/lib/python3.6/site-packages/numpy/f2py/f2py2e.py", line 633, in run_compile
    setup(ext_modules=[ext])
  File "/home/rnwatanabe/miniconda3/lib/python3.6/site-packages/numpy/distutils/core.py", line 169, in setup
    return old_setup(**new_attr)
  File "/home/rnwatanabe/miniconda3/lib/python3.6/distutils/core.py", line 148, in setup
    dist.run_commands()
  File "/home/rnwatanabe/miniconda3/lib/python3.6/distutils/dist.py", line 955, in run_command

RuntimeError: f2py failed, see output

In [4]:
%f2py_help --fcompiler

Gnu95FCompiler instance properties:
  archiver        = ['/usr/bin/gfortran', '-cr']
  compile_switch  = '-c'
  compiler_f77    = ['/usr/bin/gfortran', '-Wall', '-g', '-ffixed-form', '-
                    fno-second-underscore', '-fPIC', '-O3', '-funroll-loops']
  compiler_f90    = ['/usr/bin/gfortran', '-Wall', '-g', '-fno-second-
                    underscore', '-fPIC', '-O3', '-funroll-loops']
  compiler_fix    = ['/usr/bin/gfortran', '-Wall', '-g', '-ffixed-form', '-
                    fno-second-underscore', '-Wall', '-g', '-fno-second-
                    underscore', '-fPIC', '-O3', '-funroll-loops']
  libraries       = ['gfortran']
  library_dirs    = ['/usr/lib/gcc/x86_64-linux-gnu/5', '/usr/lib/gcc/x86_64
                    -linux-gnu/5']
  linker_exe      = ['/usr/bin/gfortran', '-Wall', '-Wall']
  linker_so       = ['/usr/bin/gfortran', '-Wall', '-g', '-Wall', '-g', '-
                    shared']
  object_switch   = '-o '
  ranlib          = ['/usr/bin/gfortran']
  ver